The script needs to be run under linux mode.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
pd.set_option('display.max_rows', 320)

from Bio import Seq
from Bio.Seq import Seq
from Bio import Restriction as Res
from Bio.SeqUtils import MeltingTemp as mt
import math
import itertools
import time
import nupack as npk

In [ ]:
# https://www.genscript.com/tools/codon-frequency-table

# Dictionary containing sense codons for amino acids. For each aa, first list
# contains cumulative probability of each codon occurence. Second list contains
# all possible codons for each aa. Third list only contains codons which occur at probability of 0.2 or greater. 
# The fourth list is the corresponding filtered codons.
sense_5to3_stats = {
    'R': ([.08, .11, .18, .20, .21, .21],
          ['cgt', 'cga', 'cgc', 'cgg', 'agg', 'aga'],
          [.21, .21, .20],
          ['aga', 'agg','cgg']), 
    'H': ([.42, .58],
          ['cat', 'cac'], 
          [.58, .42],
          ['cac', 'cat']),
    'K': ([.43, .57],
          ['aaa', 'aag'],
          [.57, .43],
          ['aag', 'aaa']),
    'D': ([.46, .54],
          ['gat', 'gac'],
          [.54, .46],
          ['gac', 'gat']),
    'E': ([.42, .58],
          ['gaa', 'gag'],
          [.58, .42],
          ['gag', 'gaa']),
    'S': ([.05, .15, .15, .19, .22, .24],
          ['tcg', 'tca', 'agt', 'tct', 'tcc', 'agc'],
          [.24, .22],
          ['agc', 'tcc']),
    'T': ([.25, .28, .36],
          ['act', 'aca', 'acc'],
          [.36, .28, .25],
          ['acc', 'aca', 'act']),
    'N': ([.47, .53],
          ['aat', 'aac'],
          [.53, .47],
          ['aac', 'aat']),
    'Q': ([.27, .73],
          ['caa', 'cag'],
          [.73, .27],
          ['cag', 'caa']),
    'C': ([.46, .54],
          ['tgt', 'tgc'],
          [.54, .46],
          ['tgc', 'tgt']),
    'G': ([.16, .25, .25, .34],
          ['ggt', 'gga', 'ggg', 'ggc'],
          [.34, .25, .25],
          ['ggc', 'gga', 'ggg']),
    'P': ([.11, .28, .29, .32],
          ['ccg', 'cca', 'cct', 'ccc'],
          [.32, .29, .28],
          ['ccc', 'cct', 'cca']),
    'A': ([.11, .23, .27, .4],
          ['gcg', 'gca', 'gct', 'gcc'],
          [.4, .27, .23],
          ['gcc', 'gct', 'gca']),
    'I': ([.17, .36, .47],
          ['ata', 'att', 'atc'],
          [.47, .36],
          ['atc', 'att']),
    'L': ([.07, .08, .13, .13, .2, .4],
          ['cta', 'tta', 'ttg', 'ctt', 'ctc', 'ctg'],
          [.4, .2],
          ['ctg', 'ctc']),
    'M': ([1],
          ['atg'],
          [1],
          ['atg']),
    'F': ([.46, .54],
          ['ttt', 'ttc'],
          [.54, .46],
          ['ttc', 'ttt']),
    'W': ([1],
          ['tgg'],
          [1],
          ['tgg']),
    'Y': ([.44, .56],
          ['tat', 'tac'],
          [.56, .44],
          ['tac', 'tat']),
    'V': ([.12, .18, .24, .46],
          ['gta', 'gtt', 'gtc', 'gtg'],
          [.46, .24],
          ['gtg', 'gtc'])}

## Input for the list of antigens

The input file should be an .xlsx Excel spreadsheet with the following columns:
1. the amino acid sequences of the antigens in single letter format, e.g. "PSPSMGRDIKVQFQS"
2. optional information each stored in a column, such as "Allele", "Disease", "Protein", "Position on the protein" etc.

In [ ]:
file_path = input("Please enter the file path for the antigen list: ")

In [ ]:
## Get the peptide list
oriFile = pd.DataFrame(pd.read_excel(file_path))

In [ ]:
# clean up file and only show info we need
cleanUp = oriFile[['Allele', 'Peptide', 'Disease', 'Protein', 'Position']]
cleanUp['len_peptide'] = cleanUp['Peptide'].str.len()
cleanUp.reset_index(inplace=True, drop=True)
#cleanUp.rename(columns={'Position':'pos'}, inplace=True)

## Functions for primer auto-generation and optimization

In [ ]:
# Initialize a dataframe that contains peptide info and a place to store the primers we're about to generate
primer_list = cleanUp.copy()
primer_list["antigen_dna"] = ""
primer_list["for_primer"] = ""
primer_list["rev_primer"] = ""
primer_list.reset_index()

In [ ]:
# Input constant regions for Ii and Linker 2, and the size of the primers to be generated
Ii = 'AAAACCTGAGGATGAAGCTGCCTAAACCTCCA'
Ii = Ii.lower()
L2 = 'GGTGGAGGAAGTTCCGGAGGTGG'
L2 = L2.lower()

In [ ]:
# Define primer properties: calculate forward and reverse primer lengths, and the variable region length of each primer.
def primer_prop(peptide, hyb_len):
    frag_len = len(Ii) + len (L2) + len(peptide)*3
    for_primer_len = (frag_len + hyb_len)//2
    rev_primer_len = frag_len + hyb_len - for_primer_len
    primer_for_add = for_primer_len - len(Ii)
    primer_rev_add = rev_primer_len - len(L2)
    
    return primer_for_add, primer_rev_add, frag_len

In [ ]:
# Given a single peptide, reverse translate to human codons with codon optimization (highest frequency in human)
def revTrans(peptide):
    sense_dna_len = len(peptide)*3
    sense_dna = ''
    for i in range(len(peptide)):
        aa = peptide[i]
        ref = sense_5to3_stats[aa]
        sense_codon = ref[3][0]
        sense_dna += sense_codon
        
    return sense_dna, sense_dna_len

In [ ]:
# Generate a forward and reverse primer based on a sense dna sequence
def primerGen(dna):
    
    # Generating the forward primer
    for_primer_str = Ii + dna[0:primer_for_add]
    for_primer = Seq(for_primer_str)
    
    # Generating the reverse primer
    rev_primer_str = dna[-primer_rev_add:] + L2
    rev_primer = Seq(rev_primer_str)
    rev_primer = rev_primer.reverse_complement()
    
    return for_primer, rev_primer

In [ ]:
## This is not necessary as we switched to Gibson assembly to insert the antigen fragment
## instead of using restriction enzymes. 
## Checkpoint 1: Check if the peptide DNA fragment (including constant regions) contains restriction enzyme cut sites
## input dna as a string
def checkpoint1(dna, peptide):
    score = 0
    sense_dna_Seq = Seq(dna)
    Bsu36I_result = Res.Bsu36I.search(sense_dna_Seq)
    BspEI_result = Res.BspEI.search(sense_dna_Seq)
    if Bsu36I_result != [] or BspEI_result != []:
        print("For peptide: '%s'" %peptide)
        print("Bsu36I or BspEI cut site is observed in the peptide DNA region!")
    else:
        score = 1
    
    return score

In [ ]:
## Checkpoint 2: check the Tm of the 20 bp hybridization region using Nearest Neighbor thermodynamics

# The prediction algorithm used by SnapGene predicts the Tm of the 20 bp hybridiztion region to be 
# 4-5C higher than the Tm_NN method from BioPyhton. 
# Updating the NN-table with new parameters used by SnapGene doesn't help much. 
# Hence, we add 4C to all the melting temperatures predicted by Tm_NN 
# to check if any of them falls beyond the range 50-69C in which our current PCR recipe works 
# (i.e. annealing temp).

# input for_primer as a seq
def checkpoint2(for_primer, peptide, hyb_len):
    score = 0
    sense_dna_len = len(peptide)*3
    hyb_region = for_primer[-hyb_len:]
    Tm = mt.Tm_NN(hyb_region) + 4
    if Tm < 50 or Tm > 69: 
        print("For peptide: '%s'" %peptide)
        print("Tm is %f, beyond the range 50-69!" %Tm)
    else:
        print('Tm is %f' %Tm)
        score = 1
    
    return score

In [ ]:
# Change the melting temperature of the 20bp hybridization region to between 50-69 by changing codons
def changeTm(peptide, dna, for_primer, rev_primer):
    # test different codon combinations for the aa in the hybridized region, 
    # excluding the ends of the hybridization region which are inluded in the hairpin structure changes
    pos_start = math.ceil((frag_len - len(rev_primer) - len(Ii))/3)
    pos_end = math.floor((len(for_primer) - len(Ii))/3)
    
    aa_pos_change = range(pos_start, pos_end)
    start_pos = 0
    flag = 'CP2'

    # generate complete codon info
    aa_to_swap, num_aa = createCodonInfo(aa_pos_change, peptide, start_pos)
    
    # generate codon combinations and pass through checkpoint 2
    # to find hybridization region with a Tm between 50-69C
    FOR, REV, dna = genCodonComb(aa_to_swap, num_aa, dna, peptide, flag)
    
    return FOR, REV, dna

In [ ]:
# Test primers with checkpoint3 and optimize primers if failed 
def checkpoint3(for_primer, rev_primer, peptide, sense_dna, hyb_len):
    primer_FOR, primer_REV = '', ''
    flag = 'CP23'
    score, message = checkpoint3a(for_primer, rev_primer, peptide)
    if score != 2:                              # didn't pass CP #3a
        print("Checkpoint 3a: Check above!")
        primer_FOR, primer_REV = 'Checkpoint 3a', '%s' %message
    else:                                       # passed CP #1, 2, and 3a
        _, p2, hyb_struc = checkpoint3b_16bphyb(for_primer, rev_primer, peptide, hyb_len)
        '''
        if p1 != True:
            print("Checkpoint 3b part 1: Check above!")
        '''
        if p2 != True:                          # didn't pass CP #3b
            aa_to_swap_for, num_aa_for, aa_to_swap_rev, num_aa_rev = codonToChange(hyb_struc, peptide)
            print('aa_to_swap_for')
            print(aa_to_swap_for)
            print('aa_to_swap_rev')
            print(aa_to_swap_rev)
            if aa_to_swap_for == [] and aa_to_swap_rev == []:
                primer_FOR, primer_REV = 'Less than 16 bp hybridized at 72C!', ''
            elif aa_to_swap_for != [] and aa_to_swap_rev != []:
                    print('change both primers')
                    primer_FOR, primer_REV, sense_dna = genCodonComb(aa_to_swap_for, num_aa_for, 
                                                                     sense_dna, peptide, flag, 
                                                                     hairpin_in_both_primers_72C=True)
                    print('change rev primer')
                    print('sense_dna after removing hairpin in forward primer: %s' %sense_dna)
                    if sense_dna == '':
                        return
                        print('did this return work for if loop: you shouldnt see this')
                    else: 
                        primer_FOR, primer_REV, sense_dna = genCodonComb(aa_to_swap_rev, num_aa_rev, 
                                                                         sense_dna, peptide, flag) 
            elif aa_to_swap_for != []:
                    primer_FOR, primer_REV, sense_dna = genCodonComb(aa_to_swap_for, num_aa_for, 
                                                                     sense_dna, peptide, flag)
                    print ('sense_dna: %s' %sense_dna)
            else:
                    print ('sense_dna: %s' %sense_dna)
                    primer_FOR, primer_REV, sense_dna = genCodonComb(aa_to_swap_rev, num_aa_rev, 
                                                                     sense_dna, peptide, flag) 
                
        else:                                   # passed CP #1, 2, 3a and 3b
            primer_FOR, primer_REV, sense_dna = for_primer, rev_primer, sense_dna
    
    return str(primer_FOR), str(primer_REV), str(sense_dna)

In [ ]:
## Checkpoint 3a: check using NUPACK at 55C if two primers (1uM) have at least 16bp annealed at the desired location; 
## AND if the annealed primers are the majority.
def checkpoint3a(FOR, REV, peptide):
    message = ''
    score = 0
    '''
    define a function
    '''
    model_55 = npk.Model(material='DNA', celsius = 55)
    FOR_str = str(FOR)
    REV_str = str(REV)
    A = npk.Strand(FOR_str, name = 'A')
    B = npk.Strand(REV_str, name = 'B')
    t1 = npk.Tube(strands = {A: 7.5e-7, B: 7.5e-7}, 
                  complexes = npk.SetSpec(max_size = 2),
                  name = 't1')
    result_55 = npk.tube_analysis(tubes = [t1], model = model_55,
                              compute = ['pfunc', 'mfe'])

   # Check at 55C if the majority of the 2 primers are annealed together.
   # Complex concetration prediction in this version (Nupack 4.0) has a small difference 
   # compared to the Nupack website prediction results.
    for key, value in result_55[t1].complex_concentrations.items():
        if key.name == '(A+B)':
            print('\n55C value')
            print(value)
            if value < 5e-7:
                print("For peptide: '%s'" %peptide)
                print('Majority of the complexes at 55C are not anneealed!')
                print(key.name)
                print(value)
                message += 'Majority of the complexes at 55C are not anneealed!'
            else:
                score += 1          

    # check if the structure contains >=16 bp hybridization region regardless of hairpin structures 
    '''
    Experimental results: 16-20 bp hybridization works. 
    '''
    AB_55 = result_55['(A+B)']
    AB_55_struc = str(AB_55.mfe[0].structure)
    print('55C struc')
    print(AB_55_struc)
    for_annealed = '(((((((((((((((('          # rev_annealed = ')))))))))))))))))'
    if for_annealed not in AB_55_struc[frag_len-len(REV):(len(FOR)+1)]:
        print("For peptide: '%s'" %peptide)
        print('Less than 16 bp annealed!')
        message += 'Less than 16 bp annealed!'
    else:
        score += 1
    
    return score, message

In [ ]:
## Checkpoint 3b: check using NUPACK at 72C if two primers (1uM) have at least 16 bp 
## annealed at the desired location; 
## AND if hairpin structure exists that might hinder the extension process.
def checkpoint3b_16bphyb(FOR, REV, peptide, hyb_len, check_for_primer_first = False):
    score = 0
    part1 = False
    part2 = False
    model_72 = npk.Model(material='DNA', celsius = 72)
    FOR_str = str(FOR)
    REV_str = str(REV)
    A = npk.Strand(FOR_str, name = 'A')
    B = npk.Strand(REV_str, name = 'B')
    t1 = npk.Tube(strands = {A: 7.5e-7, B: 7.5e-7}, 
                  complexes = npk.SetSpec(max_size = 2),
                  name = 't1')
    result_72 = npk.tube_analysis(tubes = [t1], model = model_72,
                              compute = ['pfunc', 'mfe'])
    
    # check using NUPACK at 72C to determine if a minimum of 16 bp is hybridized
    # check using NUPACK at 72C if a hairpin structure forms which might hinder the extension process
    for_con = len(FOR_str) - hyb_len
    rev_con = len(REV_str) - hyb_len
    
    # initialize the constants for primer hybridization structures
    for_1 = for_con + 1
    for_2 = for_con + 2
    for_3 = for_con + 3
    for_4 = for_con + 4
    
    rev_1 = rev_con + 1
    rev_2 = rev_con + 2
    rev_3 = rev_con + 3
    rev_4 = rev_con + 4
    
    hyb_1 = hyb_len - 1
    hyb_2 = hyb_len - 2
    hyb_3 = hyb_len - 3
    hyb_4 = hyb_len - 4
    
    perfect_struc = npk.Structure(f'.{for_con}({hyb_len}+.{rev_con}){hyb_len}')
    # 1 bp unhybridized
    unhyb_1bp_1 = npk.Structure(f'.{for_1}({hyb_1}+.{rev_con}){hyb_1}.1')
    unhyb_1bp_2 = npk.Structure(f'.{for_con}({hyb_1}.1+.{rev_1}){hyb_1}')
    # 2 bp unhybridized
    unhyb_2bp_1 = npk.Structure(f'.{for_2}({hyb_2}+.{rev_con}){hyb_2}.2')
    unhyb_2bp_2 = npk.Structure(f'.{for_con}({hyb_2}.2+.{rev_2}){hyb_2}')
    unhyb_2bp_3 = npk.Structure(f'.{for_1}({hyb_2}.1+.{rev_1}){hyb_2}.1')
    # 3 bp unhybridized
    unhyb_3bp_1 = npk.Structure(f'.{for_3}({hyb_3}+.{rev_con}){hyb_3}.3')
    unhyb_3bp_2 = npk.Structure(f'.{for_con}({hyb_3}.3+.{rev_3}){hyb_3}')
    unhyb_3bp_3 = npk.Structure(f'.{for_2}({hyb_3}.1+.{rev_1}){hyb_3}.2')
    unhyb_3bp_4 = npk.Structure(f'.{for_1}({hyb_3}.2+.{rev_2}){hyb_3}.1')
    # 4 bp unhybridized
    unhyb_4bp_1 = npk.Structure(f'.{for_4}({hyb_4}+.{rev_con}){hyb_4}.4')
    unhyb_4bp_2 = npk.Structure(f'.{for_con}({hyb_4}.4+.{rev_4}){hyb_4}')
    unhyb_4bp_3 = npk.Structure(f'.{for_3}({hyb_4}.1+.{rev_1}){hyb_4}.3')
    unhyb_4bp_4 = npk.Structure(f'.{for_1}({hyb_4}.3+.{rev_3}){hyb_4}.1')
    unhyb_4bp_5 = npk.Structure(f'.{for_2}({hyb_4}.2+.{rev_2}){hyb_4}.2')
             
    ideal_struc_list = [perfect_struc, 
                        unhyb_1bp_1, unhyb_1bp_2, 
                        unhyb_2bp_1, unhyb_2bp_2, unhyb_2bp_3,
                        unhyb_3bp_1, unhyb_3bp_2, unhyb_3bp_3, unhyb_3bp_4,
                        unhyb_4bp_1, unhyb_4bp_2, unhyb_4bp_3, unhyb_4bp_4, unhyb_4bp_5]
    # 16-mer peptide, max 17 bp hybridized, min 16 bp hybridized
    ideal_struc_list_16mer = [perfect_struc, 
                              unhyb_1bp_1, unhyb_1bp_2]
    
    AB_72 = result_72['(A+B)']
    AB_72_struc = AB_72.mfe[0].structure
    print('72C struc')
    print(AB_72_struc)
    print('FOR: %s' %FOR_str)
    print('REV: %s' %REV_str)
      
    if check_for_primer_first == True:
        if str(npk.Structure(f'.{for_con}')) in str(AB_72_struc)[:for_con]:
            score += 1
            return score, part2, AB_72_struc
        else:
            return score, part2, AB_72_struc
    
    if len(peptide) == 16:
        ideal_struc = ideal_struc_list_16mer
    else:
        ideal_struc = ideal_struc_list
        
    if AB_72_struc not in ideal_struc:
        print('Hairpin structure or < 16 bp unhybridized regions might exist in primer at 72C!')
    else:
        part2 = True
        score += 1
    
    return score, part2, AB_72_struc

In [ ]:
# identify the position of the amino acid in the peptide list, whose codon will be changed
def codonPos(aa_num, dna_struc_slice):
    aa_pos_change = []
    for i in range(aa_num):
        pos_end = (i+1)*3 - 1
        codon = dna_struc_slice[pos_end-2:pos_end+1]
        if ')' in codon or '(' in codon:
            aa_pos_change.append(i)
 
    return aa_pos_change

In [ ]:
# create an array of peptides whose codon will be changed, info includes: 
# name of aa, candidate codons, swap_index for generating different combinations
# of codons, and the codon location in the sense dna that will be changed
def createCodonInfo(aa_pos_change, peptide, start_pos):
    aa_to_swap = []
    for i in aa_pos_change:
        aa = {}
        aa['name'] = peptide[i + start_pos]
        aa['candidates'] = sense_5to3_stats[aa['name']][3] # I'm allowing the _reverse primer: first bp of the first codon 
        aa['swap_index'] = 0                               # OR forward primer: last bp of last codon_ to be changed as well, 
        aa['index_sense_dna'] = (i+start_pos)*3            # which is in hybridization region, 
        aa_to_swap.append(aa)                              # this will not change the result in Checkpoint #2 (55C annealing).
    num_aa = len(aa_to_swap) 
    
    return aa_to_swap, num_aa

In [ ]:
# Gather info needed to remove hairpin in 72C primer struc
def codonToChange(struc_72C, peptide):
    # Identify editable positions that are outside the constant region and hybridization region
    struc_72C = str(struc_72C) 
    print('check struc_72C')
    print(struc_72C)
    stop_for = frag_len - len(rev_primer)
    struc_for = struc_72C[len(Ii):stop_for]
    aa_num_for = math.ceil(len(struc_for)/3)
    
    start_rev = len(for_primer) + 1 + len(L2)
    struc_rev = struc_72C[start_rev:-hyb_len]
    rev_pos = struc_rev[::-1]
    aa_num_rev = math.ceil(len(struc_rev)/3)
    # allowing to change the end of the hybridization sites
    if (3-len(struc_rev)%3) == 1:
        rev_pos_added = '.' + rev_pos
    elif (3-len(struc_rev)%3) == 2:
        rev_pos_added = '..' + rev_pos
        
    # Define the starting position of the corresponding amino acid in the peptide
    start_pos_for = 0
    start_pos_rev = math.floor((len(for_primer) - len(Ii))/3)

    # codon to be changed in forward/reverse primer
    aa_pos_change_for = codonPos(aa_num_for, struc_for)
    aa_pos_change_rev = codonPos(aa_num_rev, rev_pos_added)

    # information needed for changing codons   
    aa_to_swap_for, num_aa_for = createCodonInfo(aa_pos_change_for, peptide, start_pos_for)
    aa_to_swap_rev, num_aa_rev = createCodonInfo(aa_pos_change_rev, peptide, start_pos_rev)
    
    return aa_to_swap_for, num_aa_for, aa_to_swap_rev, num_aa_rev

In [ ]:
# Generate different codon combinations on a selected set amino acids
def genCodonComb(aa_to_swap, num_aa, dna, peptide, flag, hairpin_in_both_primers_72C = False):
    total_iter = 1
    
    for i in range(num_aa):
        total_iter *= len(aa_to_swap[i]['candidates'])
    
    print('here')
    print('total_iter %d' %total_iter)
    
    for i in range(total_iter):
         # generate new seq
        for j in range(num_aa):
            amino_acid = aa_to_swap[j]
            pos = amino_acid['index_sense_dna']
            swap = amino_acid['swap_index']
            dna = dna[:pos] + amino_acid['candidates'][swap] + dna[pos+3:]
        
        # generate new forward and reverse primers using the new sense DNA sequence
        FOR, REV = primerGen(dna)
        print('FOR_test: %s' %FOR)
        print('REV_test: %s' %REV)
        
        # determine which checkpoints to check
        if flag == 'CP23':
            message = 'Unable to find optimized primers!'
            # check through Checkpoint 2, 3a, and 3b
            score3a, _ = checkpoint3a(FOR, REV, peptide)
            score3b, _, _ = checkpoint3b_16bphyb(FOR, REV, peptide, hyb_len)
            if hairpin_in_both_primers_72C == True:
                print('here-1')
                score3b, _, _ = checkpoint3b_16bphyb(FOR, REV, peptide, hyb_len, check_for_primer_first = True)
                if checkpoint2(FOR, peptide, hyb_len) + score3a + score3b == 4:
                    print('here-2')
                    return FOR, REV, dna
                    break
            if checkpoint2(FOR, peptide, hyb_len) + score3a + score3b == 4:
                print('Sense_dna: %s' %dna)
                print('Forward primer: %s' %FOR)
                print('Reverse primer: %s' %REV)
                return FOR, REV, dna
                break
        elif flag == 'CP2':
            message = 'Unable to make Tm in the 50-69C range!'
            # check through checkpoint 2
            if checkpoint2(FOR, peptide, hyb_len) == 1:
                print('Checkpoint 2 passed.')
                return FOR, REV, dna
                break
        
        # increment the swap index
        for j in range(num_aa):
            aa_to_swap[j]['swap_index'] += 1
            if aa_to_swap[j]['swap_index'] == len(aa_to_swap[j]['candidates']):
                aa_to_swap[j]['swap_index'] = 0
            else:
                break
        
        if i == total_iter - 1:
            print("%s" %message)
            FOR, REV, dna = '%s' %message, '', ''
            return FOR, REV, dna

## Generate and optimize primers

In [ ]:
# Generate forward and reverse primers for each peptide; 
# Check if the primers satisfy the following criteria: checkpoint 1, 2 and 3.

for index in primer_list.index:
    
    print('\nPos %d' %index)
        
    peptide = primer_list.loc[index, 'Peptide']
    if len(peptide) == 16:
        hyb_len = 17
    else:
        hyb_len = 20
    sense_dna, sense_dna_len = revTrans(peptide)
    total_dna = Ii[-10:] + sense_dna + L2[:10]
    
    # overall workflow

    primer_for_add, primer_rev_add, frag_len = primer_prop(peptide, hyb_len)
    for_primer, rev_primer = primerGen(sense_dna)
    
    # If using Gibson assembly, then checkpoint 1 is not necessary.
    # CP #1 is only needed if double digestion using restriction enzymes are used for inserting the antigen fragments 
    # into the pcDNA plasmid.
    '''
    if checkpoint1(total_dna, peptide) != 1:        # didn't pass CP #1
        print("Checkpoint 1: Check above!")
        primer_FOR, primer_REV = 'Checkpoint 1', ''
    '''
    if checkpoint2(for_primer, peptide, hyb_len) != 1:     # passed CP #1, but didn't pass CP #2
        for_primer, rev_primer, sense_dna = changeTm(peptide, sense_dna, for_primer, rev_primer)
        if for_primer != 'Unable to make Tm in the 50-69C range!':
            primer_FOR, primer_REV, sense_dna = checkpoint3(for_primer, rev_primer, 
                                                            peptide, sense_dna, hyb_len)        
        else:
            primer_FOR, primer_REV = 'Unable to make Tm in the 50-69C range!', ''
    else:                                           # passed CP #1 and 2
        print("for_primer %s" %for_primer)
        print("rev_primer %s" %rev_primer)
        primer_FOR, primer_REV, sense_dna = checkpoint3(for_primer, rev_primer, 
                                                        peptide, sense_dna, hyb_len)
    
    print(sense_dna)
    primer_list.loc[index, 'antigen_dna'] = str(sense_dna)
    primer_list.loc[index, 'for_primer'] = str(primer_FOR)
    primer_list.loc[index, 'rev_primer'] = str(primer_REV)

## Ouptut the optimized primers

In [ ]:
primer_list['ID'] = primer_list.index + 1
primer_list['for_primer_name'] = 'DR_DP_FOR_' + primer_list['ID'].astype(str)
primer_list['rev_primer_name'] = 'DR_DP_REV_' + primer_list['ID'].astype(str)

In [ ]:
output_path = input("Please enter the file path for the output file: ")

In [ ]:
# output the primers generated. Need to manually check the ones with error messages.
primer_list.to_excel(output_path)